<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/over_%EC%A0%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()

In [12]:
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [7]:
#데이터 세트
%%bigquery --project ringed-marker-347607
select *
from ch3.popular_products

,product_id,category,score
0,D003,drama,78
1,D001,drama,90
2,D004,drama,58
3,D002,drama,82
4,A003,action,78
5,A002,action,81
6,A001,action,94
7,A004,action,64


#over(partition by())

In [11]:
%%bigquery --project ringed-marker-347607
select category,
  (select sum(score) from ch3.popular_products) as score
from ch3.popular_products
# 서브쿼리를 사용해 합계를 계산, 하지만 지저분 하고 카테고리 별로 계산하지 못함

,category,score
0,drama,625
1,drama,625
2,drama,625
3,drama,625
4,action,625
5,action,625
6,action,625
7,action,625


In [14]:
%%bigquery --project ringed-marker-347607
select category,sum(score)over()
from ch3.popular_products
#over 절을 사용하여 깔끔하지만 카테고리별로 계산하지 못함

,category,f0_
0,action,625
1,action,625
2,action,625
3,drama,625
4,drama,625
5,drama,625
6,drama,625
7,action,625


In [15]:
%%bigquery --project ringed-marker-347607
select category,sum(score)over(partition by category)
from ch3.popular_products
#partition by 를 사용하여 성공

,category,f0_
0,drama,308
1,drama,308
2,drama,308
3,drama,308
4,action,317
5,action,317
6,action,317
7,action,317


#over(order by))

In [17]:
%%bigquery --project ringed-marker-347607
select
  product_id,
  score,
  row_number() over(order by score desc) as row_number,
  rank() over(order by score desc) as rank,
  dense_rank() over(order by score desc) as dense_rank,
  lag(product_id) over(order by score desc) as lag1,
  lag(product_id,2) over (order by score desc) as lag2,
  lead(product_id) over (order by score desc) as lead,
  lead(product_id,2) over (order by score desc) as lead2
from ch3.popular_products
order by row_number


,product_id,score,row_number,rank,dense_rank,lag1,lag2,lead,lead2
0,A001,94,1,1,1,None,None,D001,D002
1,D001,90,2,2,2,A001,None,D002,A002
2,D002,82,3,3,3,D001,A001,A002,D003
3,A002,81,4,4,4,D002,D001,D003,A003
4,D003,78,5,5,5,A002,D002,A003,A004
5,A003,78,6,5,5,D003,A002,A004,D004
6,A004,64,7,7,6,A003,D003,D004,None
7,D004,58,8,8,7,A004,A003,None,None
